# Data Scraping and Cleaning


In [14]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
import tabula
import pycountry
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geopandas as gpd

warnings.filterwarnings(action = 'ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### 1. Data Source: [ROCS](http://rocs.hu-berlin.de/viz/sgb/)

- Airport name and three-letter symbol
- Country
- Passenger flux per day
- Connected airports
- Effective distance to outbreak center
- Geodesic distance to outbreak center (km)

In [2]:
response1 = requests.get('http://rocs.hu-berlin.de/viz/sgb/airports0.95_2014/airports.json')
rocs_airport_info_df = pd.DataFrame(response1.json()['nodes'])

response2 = requests.get('http://rocs.hu-berlin.de/viz/sgb/airports0.95_2014/spt_673.json')
rocs_data_df = pd.DataFrame(response2.json())  

# Join on index
rocs_df = rocs_data_df.join(rocs_airport_info_df, how= 'outer')
rocs_df2 = rocs_df[['Dfrom', 'Dgeo', 'name', 'continent_name', 'region_name', 'country_name', 'lid', 'F',
                    'k', 'lon', 'lat']]

rocs_df2.rename(columns = {'Dfrom':'effective_dist', 'Dgeo':'geo_dist', 'name':'airport_name', 
                     'continent_name':'continent_name', 'region_name':'region_name', 'country_name':'country_name',
                    'lid':'airport_id', 'F':'ann_pass_flux', 'k':'connected_airports', 'lon':'long', 'lat':'lat'},
               inplace = True)
rocs_df2.head()

,effective_dist,geo_dist,airport_name,continent_name,region_name,country_name,airport_id,ann_pass_flux,connected_airports,long,lat
0,9.626587,8111.034829,Addis Ababa,Africa,Eastern Africa,Ethiopia,ADD,3887132.5,56.5,38.799444,8.977778
1,13.897219,7763.569189,Asmara,Africa,Eastern Africa,Eritrea,ASM,307820.5,10.5,38.910556,15.291944
2,14.024414,9712.495427,Bujumbura,Africa,Eastern Africa,Burundi,BJM,252165.0,6.5,29.318611,-3.323889
3,13.695018,8990.314878,Dar es Salaam,Africa,Eastern Africa,Tanzania United Republic of,DAR,1840029.0,17.0,39.202500,-6.878056
4,13.038729,9221.694455,Entebbe,Africa,Eastern Africa,Uganda,EBB,1079933.0,16.5,32.443611,0.042500


In [3]:
# Rename country to normalized convention 

def retrieve_country(val):
    try:
        spec = pycountry.countries.search_fuzzy(val)
        search_results = spec[0].name
    except:
        search_results = np.nan
    return search_results

rocs_df2['test_country'] = rocs_df2.country_name.map(lambda val: retrieve_country(val))

rocs_df2.test_country.isna().value_counts()

False    1240
True       52
Name: test_country, dtype: int64

In [4]:
for index, val in enumerate(rocs_df2.test_country):
    if val is np.nan:
        print(index, rocs_df2.country_name[index])

3 Tanzania United Republic of
6 Tanzania United Republic of
13 Tanzania United Republic of
19 Tanzania United Republic of
23 Congo Democratic Republic of
33 Libyan Arab Jamahiriya
46 Libyan Arab Jamahiriya
54 Libyan Arab Jamahiriya
57 Libyan Arab Jamahiriya
75 Cape Verde
90 Cape Verde
92 Cape Verde
93 Antigua and Barbuda, Leeward Islands
103 Grenada, Windward Islands
118 Saint Kitts and Nevis, Leeward Islands
121 Virgin Islands, US
122 Virgin Islands, US
123 St Maarten (Dutch Part)
527 Korea Republic of
528 Korea Republic of
549 Korea Republic of
557 Hong Kong (SAR) China
568 Korea Republic of
596 Korea Republic of
600 Korea Republic of
612 Macao (SAR) China
635 Korea Republic of
638 Korea Republic of
648 Korea Republic of
667 Korea Republic of
697 Iran Islamic Republic of
698 Iran Islamic Republic of
703 Iran Islamic Republic of
705 Iran Islamic Republic of
719 Iran Islamic Republic of
720 Iran Islamic Republic of
736 Iran Islamic Republic of
737 Iran Islamic Republic of
742 Iran Isla

In [5]:
country_map = {'Tanzania United Republic of':'Tanzania',
                                   'Congo Democratic Republic of':'Congo, The Democratic Republic of the',
                                   'Libyan Arab Jamahiriya':'Libya',
                                   'Cape Verde':'Cabo Verde',
                                   'Antigua and Barbuda, Leeward Islands': 'Antigua and Barbuda',
                                   'Grenada, Windward Islands':'Grenada',
                                   'Saint Kitts and Nevis, Leeward Islands': 'Saint Kitts and Nevis',
                                   'Virgin Islands, US': 'Virgin Islands, U.S.',
                                   'St Maarten (Dutch Part)':'Sint Maarten (Dutch part)',
                                   'Korea Republic of': 'Korea, Republic of',
                                   'Hong Kong (SAR) China': 'Hong Kong',
                                   'Macao (SAR) China':'Macao',
                                   'Iran Islamic Republic of':'Iran, Islamic Republic of',
                                   'Moldova Republic of':'Moldova',
                                   'Ireland Republic of':'Ireland',
                                   'Macedonia Former Yugoslav Republic of':'North Macedonia',
                                   'Northern Mariana Islands (except Guam)':'Northern Mariana Islands'}

country_zip = list(zip(rocs_df2.country_name, rocs_df2.test_country))

for i, (CN, TC) in enumerate(country_zip):
    if TC is np.nan:
        rocs_df2.test_country[i] = country_map[CN]

rocs_df2.test_country.isna().value_counts()

False    1292
Name: test_country, dtype: int64

In [6]:
rocs_df2.head()

,effective_dist,geo_dist,airport_name,continent_name,region_name,country_name,airport_id,ann_pass_flux,connected_airports,long,lat,test_country
0,9.626587,8111.034829,Addis Ababa,Africa,Eastern Africa,Ethiopia,ADD,3887132.5,56.5,38.799444,8.977778,Ethiopia
1,13.897219,7763.569189,Asmara,Africa,Eastern Africa,Eritrea,ASM,307820.5,10.5,38.910556,15.291944,Eritrea
2,14.024414,9712.495427,Bujumbura,Africa,Eastern Africa,Burundi,BJM,252165.0,6.5,29.318611,-3.323889,Burundi
3,13.695018,8990.314878,Dar es Salaam,Africa,Eastern Africa,Tanzania United Republic of,DAR,1840029.0,17.0,39.202500,-6.878056,Tanzania
4,13.038729,9221.694455,Entebbe,Africa,Eastern Africa,Uganda,EBB,1079933.0,16.5,32.443611,0.042500,Uganda


### 2. Data Source: [US Dept of Transportation](https://data.transportation.gov/Aviation/International_Report_Passengers/xgub-n9bw)
This data only provides passenger information on international US flights.

- Year, month/datetime
- US airport and foreign airport IDs
- Number of passengers

In [7]:
response = requests.get('https://data.transportation.gov/resource/xgub-n9bw.json')
airports_df = pd.DataFrame(response.json())
airports_df.head()

,data_dte,year,month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,scheduled,charter,total
0,2019-06-01T00:00:00.000,2019,6,12478,JFK,22,12972,LHR,493,19682,VS,0,Passengers,85790,0,85790
1,2019-06-01T00:00:00.000,2019,6,12478,JFK,22,10920,CDG,427,19532,AF,0,Passengers,85324,0,85324
2,2019-06-01T00:00:00.000,2019,6,12953,LGA,22,16271,YYZ,936,19531,AC,0,Passengers,82389,0,82389
3,2019-06-01T00:00:00.000,2019,6,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,79975,0,79975
4,2019-06-01T00:00:00.000,2019,6,12892,LAX,91,16271,YYZ,936,19531,AC,0,Passengers,73172,0,73172


In [8]:
print('Number of US Airports: ', len(airports_df.usg_apt.unique()))
print('Number of Foreign Airports: ', len(airports_df.fg_apt.unique()))

Number of US Airports:  52
Number of Foreign Airports:  132


In [9]:
airports_df.year.value_counts()

2019    1000
Name: year, dtype: int64

In [10]:
airports_df.data_dte.value_counts()

2019-06-01T00:00:00.000    1000
Name: data_dte, dtype: int64

In [11]:
airports_df.type.value_counts()

Passengers    1000
Name: type, dtype: int64

In [12]:
airports_df.scheduled = airports_df.scheduled.astype(int)
airports_df.total = airports_df.total.astype(int)

for i in range (0, len(airports_df)):
    if (airports_df.scheduled[i] == airports_df.total[i]):
        pass
    elif (airports_df.scheduled[i] != airports_df.total[i]):
        print(i, (airports_df.scheduled[i] - airports_df.total[i]))

16 -108
465 -53
521 -58


Take total passengers instead of scheduled passengers since scheduled passengers differ for at least three flights.

### 3. Data Source: [International Health Regulations SPAR Index](http://apps.who.int/gho/data/view.main.IHRSPARCTRYALLv?lang=en)
WHO International Health Regulations State Parties Self-Assessment Annual Reporting index, a combination of indicators (country-level). The latest data is available for 2018. Index values range from 0-100.

- Country
- Legislation and Financing
- IHR Coordination and National IHR Focal Point Functions
- Zoonotic Events and the Human-animal Interface
- Food Safety
- Laboratory
- Surveillance
- Human Resources
- National Health Emergency Framework
- Health Service Provision
- Risk Communication
- Points of Entry
- Chemical Events
- Radiation Emergencies

In [13]:
spar_df = pd.read_csv('../xmart.csv')
spar_df.head()

,Country,Legislation and Financing; 2018,IHR Coordination and National IHR Focal Point Functions; 2018,Zoonotic Events and the Human-animal Interface; 2018,Food Safety; 2018,Laboratory; 2018,Surveillance; 2018,Human Resources; 2018,National Health Emergency Framework; 2018,Health Service Provision; 2018,Risk Communication; 2018,Points of Entry; 2018,Chemical Events; 2018,Radiation Emergencies; 2018
0,Afghanistan,13,60,80,20,40,80,40,27,40,20,10,0,20
1,Albania,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data
2,Algeria,100,90,80,80,60,80,80,53,93,60,60,100,100
3,Andorra,47,50,60,80,67,20,20,0,53,60,20,0,0
4,Angola,47,80,80,20,60,90,60,60,47,80,40,40,60


In [14]:
int_cols = list(spar_df.columns)
int_cols.remove('Country')

for col in int_cols:
    for index, val in enumerate(spar_df[col]):
        if val == 'No data':
            spar_df[col][index] = np.nan
        elif val != 'No data':
            spar_df[col][index] = float(val)

spar_df['Index'] = spar_df.apply(lambda _: 0, axis=1)

for index in range(0, len(spar_df)):
    spar_df['Index'][index] = sum(spar_df.iloc[index][int_cols])/len(int_cols)
    
spar_df2 = spar_df[['Country', 'Index']]

# impute mean into 13 country entries
spar_df2.Index.fillna(spar_df2.Index.mean(), inplace= True)

spar_df2.Index.isna().value_counts()

False    194
Name: Index, dtype: int64

In [15]:
# Rename country to normalized convention 

spar_df2['test_country'] = spar_df2.Country.map(lambda val: retrieve_country(val))

spar_df2.test_country.isna().value_counts()

False    189
True       5
Name: test_country, dtype: int64

In [16]:
for index, val in enumerate(spar_df2.test_country):
    if val is np.nan:
        print(index, spar_df2.Country[index])

20 Bolivia (Plurinational State of)
47 Democratic Republic of the Congo
80 Iran (Islamic Republic of)
111 Micronesia (Federated States of)
189 Venezuela (Bolivarian Republic of)


In [17]:
country_map = {'Bolivia (Plurinational State of)':'Bolivia',
                                   'Democratic Republic of the Congo':'Congo, The Democratic Republic of the',
                                   'Libyan Arab Jamahiriya':'Libya',
                                   'Iran (Islamic Republic of)':'Iran, Islamic Republic of',
                                   'Micronesia (Federated States of)':'Micronesia, Federated States of',
                                   'Venezuela (Bolivarian Republic of)':'Venezuela, Bolivarian Republic of'}

country_zip = list(zip(spar_df2.Country, spar_df2.test_country))

for i, (CN, TC) in enumerate(country_zip):
    if TC is np.nan:
        spar_df2.test_country[i] = country_map[CN]

spar_df2.test_country.isna().value_counts()

False    194
Name: test_country, dtype: int64

### 4. Data Source: [Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv)
Sourcing daily cumulative confirmed cases, recoveries, and deaths per location (Jan 21 - present) from:
- World Health Organization (WHO): https://www.who.int/
- DXY.cn. Pneumonia. 2020. http://3g.dxy.cn/newh5/view/pneumonia.
- BNO News: https://bnonews.com/index.php/2020/02/the-latest-coronavirus-cases/
- National Health Commission of the People’s Republic of China (NHC):
http://www.nhc.gov.cn/xcs/yqtb/list_gzbd.shtml
- China CDC (CCDC): http://weekly.chinacdc.cn/news/TrackingtheEpidemic.htm
- Hong Kong Department of Health: https://www.chp.gov.hk/en/features/102465.html
- Macau Government: https://www.ssm.gov.mo/portal/
- Taiwan CDC: https://sites.google.com/cdc.gov.tw/2019ncov/taiwan?authuser=0
- US CDC: https://www.cdc.gov/coronavirus/2019-ncov/index.html
- Government of Canada: https://www.canada.ca/en/public-health/services/diseases/coronavirus.html
- Australia Government Department of Health: https://www.health.gov.au/news/coronavirus-update-at-a-glance
- European Centre for Disease Prevention and Control (ECDC): https://www.ecdc.europa.eu/en/geographical-distribution-2019-ncov-cases
- Ministry of Health Singapore (MOH): https://www.moh.gov.sg/covid-19

In [28]:
# git pull origin master to update COVID-19 files for most recent confirmed cases
# https://github.com/CSSEGISandData/COVID-19
jhu_df = pd.read_csv('../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
jhu_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,106,152,200,237,297,340,408,480,530,591,665,733,779,830,860,889,910,934,950,962,973,982,986,987,988,989,989,989,989,989,989
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,91,111,114,139,168,191,212,228,253,274,297,315,326,337,342,352,366,372,375,380,381,387,393,395,396,399,399,399,400,400,410
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,132,147,182,211,247,300,337,366,389,411,426,428,468,486,505,518,529,537,544,551,553,555,560,567,572,573,575,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,80,84,101,120,144,159,179,194,205,215,224,239,250,261,267,272,279,281,285,287,290,292,293,293,293,293,293,293,294,294,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,19,24,26,29,40,51,55,57,62,62,67,79,83,83,86,87,90,90,90,90,91,91,91,91,91,91,91,91,91,91,91


In [29]:
jhu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 41 columns):
Province/State    60 non-null object
Country/Region    105 non-null object
Lat               105 non-null float64
Long              105 non-null float64
1/22/20           105 non-null int64
1/23/20           105 non-null int64
1/24/20           105 non-null int64
1/25/20           105 non-null int64
1/26/20           105 non-null int64
1/27/20           105 non-null int64
1/28/20           105 non-null int64
1/29/20           105 non-null int64
1/30/20           105 non-null int64
1/31/20           105 non-null int64
2/1/20            105 non-null int64
2/2/20            105 non-null int64
2/3/20            105 non-null int64
2/4/20            105 non-null int64
2/5/20            105 non-null int64
2/6/20            105 non-null int64
2/7/20            105 non-null int64
2/8/20            105 non-null int64
2/9/20            105 non-null int64
2/10/20           105 non-null int64
2/

In [5]:
from haversine import haversine

haversine((45.7597, 4.8422),(48.8567, 2.3508), unit='mi')

243.71250609539814

In [31]:
# Rename country to normalized convention 

def retrieve_country(val):
    try:
        spec = pycountry.countries.search_fuzzy(val)
        search_results = spec[0].name
    except:
        search_results = np.nan
    return search_results

jhu_df['country'] = jhu_df['Country/Region'].map(lambda val: retrieve_country(val))

jhu_df.country.isna().value_counts()

False    71
True     34
Name: country, dtype: int64

In [32]:
for index, val in enumerate(jhu_df.country):
    if val is np.nan:
        print(index, jhu_df['Country/Region'][index])

0 Mainland China
1 Mainland China
2 Mainland China
3 Mainland China
4 Mainland China
5 Mainland China
6 Mainland China
7 Mainland China
8 Mainland China
9 Mainland China
10 Mainland China
11 Mainland China
12 Mainland China
13 Mainland China
14 Mainland China
15 Mainland China
16 Mainland China
17 Mainland China
18 Mainland China
19 Mainland China
20 Mainland China
21 Mainland China
22 Mainland China
23 Mainland China
24 Mainland China
25 Mainland China
26 Mainland China
27 Mainland China
28 Mainland China
29 Mainland China
30 Mainland China
33 South Korea
38 Macau
71 Others


In [33]:
jhu_df.loc[jhu_df['Country/Region'] == 'Others']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,country
71,Diamond Princess cruise ship,Others,35.4437,139.638,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,61,64,135,135,175,175,218,285,355,454,542,621,634,634,634,691,691,691,705,705,NaN


In [36]:
country_map = {'Mainland China':'China',
                                   'Macau':'Macao',
                                   'South Korea':'''Korea, Democratic People's Republic of''',
                                   'Others':'Diamond Princess cruise ship'}

country_zip = list(zip(jhu_df['Country/Region'], jhu_df.country))

for i, (CN, TC) in enumerate(country_zip):
    if TC is np.nan:
        jhu_df.country[i] = country_map[CN]

jhu_df.country.isna().value_counts()

False    105
Name: country, dtype: int64

False    105
Name: country, dtype: int64

### 5. Data Sources: 
### [UN, source = World Bank](http://data.un.org/Data.aspx?q=GDP+per+capita&d=WDI&f=Indicator_Code%3aNY.GDP.PCAP.PP.CD)

- GDP per capita for 2018, 230 country records

### [London Datastore, source = UN](https://data.london.gov.uk/dataset/global-city-population-estimates)

- City populations over 300k time series (1950-2030, 5 yr) with latitude/longitude

In [23]:
# GDP per capita, PPP (current international $)
gdp_df = pd.read_csv('../UNdata_Export_20200221_065525496.csv')
gdp_df.head()

,Country or Area,Year,Value,Value Footnotes
0,Afghanistan,2018,1955.006208,NaN
1,Albania,2018,13364.155397,NaN
2,Algeria,2018,15481.787620,NaN
3,Angola,2018,6452.355165,NaN
4,Antigua and Barbuda,2018,26868.133524,NaN


In [24]:
# Rename country to normalized convention 

gdp_df['test_country'] = gdp_df['Country or Area'].map(lambda val: retrieve_country(val))

gdp_df.test_country.isna().value_counts()

False    175
True      55
Name: test_country, dtype: int64

In [25]:
for index, val in enumerate(gdp_df.test_country):
    if val is np.nan:
        print(index, gdp_df['Country or Area'][index])

5 Arab World
27 CÃ´te d'Ivoire
32 Caribbean small states
34 Central Europe and the Baltics
43 CuraÃ§ao
45 Dem. Rep. Congo
49 Early-demographic dividend
50 East Asia & Pacific
51 East Asia & Pacific (excluding high income)
52 East Asia & Pacific (IDA & IBRD)
60 Euro area
61 Europe & Central Asia
62 Europe & Central Asia (excluding high income)
63 Europe & Central Asia (IDA & IBRD)
64 European Union
67 Fragile and conflict affected situations
80 Heavily indebted poor countries (HIPC)
81 High income
83 Hong Kong SAR, China
85 IBRD only
87 IDA & IBRD total
88 IDA blend
89 IDA only
90 IDA total
107 Lao PDR
108 Late-demographic dividend
109 Latin America & Caribbean
110 Latin America & Caribbean (excluding high income)
111 Latin America & Caribbean (IDA & IBRD)
113 Least developed countries: UN classification
119 Low & middle income
120 Low income
121 Lower middle income
123 Macao SAR, China
135 Middle East & North Africa
136 Middle income
151 North America
154 OECD members
156 Other small s

In [26]:
# There are 195 countries in the world according to Google, but we have 230 here! 
# Much of this is extraneous and can be dropped. Some country renaming to be done.

gdp_df = gdp_df.drop(labels=[5, 32, 34, 49, 50, 51, 52, 60, 61, 62, 63, 64, 67, 80, 81, 85, 87, 88, 89, 90,
                  108, 109, 110, 111, 113, 119, 120, 121, 135, 136, 151, 154, 156, 157, 167,
                  168, 184, 187, 188, 194, 195, 196, 220, 226], axis = 0).reset_index()

country_map = {'''CÃ´te d'Ivoire''':'''Côte d'Ivoire''',
               'CuraÃ§ao':'Curaçao',
               'Dem. Rep. Congo':'Congo, The Democratic Republic of the',
               'Hong Kong SAR, China':'Hong Kong',
               'Lao PDR':"Lao People's Democratic Republic",
              'Macao SAR, China':'Macao',
              'SÃ£o TomÃ© and Principe':'Sao Tome and Principe',
              'St. Kitts and Nevis':'Saint Kitts and Nevis',
              'St. Lucia':'Saint Lucia',
              'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines',
              'West Bank and Gaza':'Palestine, State of'}

country_zip = list(zip(gdp_df['Country or Area'], gdp_df.test_country))

for i, (CN, TC) in enumerate(country_zip):
    if TC is np.nan:
        gdp_df.test_country[i] = country_map[CN]

gdp_df.test_country.isna().value_counts()

False    186
Name: test_country, dtype: int64

In [27]:
city_pop_df = pd.read_excel('../global-city-population-estimates.xls', sheet_name=1)
city_pop_df.head()

,Country Code,Country or area,City Code,Urban Agglomeration,Note,Latitude,Longitude,1950,1955,1960,1965,1970,1975,1980,1985,1990,1995,2000,2005,2010,2015,2020,2025,2030
0,392,Japan,21671,Tokyo,325.0,35.689500,139.691710,11274.641,13712.679,16678.821,20284.371,23297.503,26614.733,28548.512,30303.794,32530.003,33586.573,34449.908,35621.544,36833.979,38001.018,38323.229,37875.951,37190.489
1,356,India,21228,Delhi,318.0,28.666670,77.216670,1369.369,1781.624,2282.962,2845.042,3530.693,4425.964,5558.481,7325.185,9725.885,12407.372,15732.304,18670.494,21935.142,25703.168,29347.622,32726.564,36060.100
2,156,China,20656,Shanghai,202.0,31.222220,121.458060,4300.942,5846.383,6819.634,6428.131,6036.492,5626.640,5966.171,6846.765,7823.028,10449.535,13958.981,16763.455,19979.977,23740.778,27137.316,29441.569,30750.671
3,76,Brazil,20287,São Paulo,NaN,-23.547500,-46.636110,2334.038,3043.828,3969.759,5494.150,7620.490,9614.016,12089.454,13394.815,14775.840,15913.473,17014.078,18288.134,19659.808,21066.245,22118.900,22899.066,23444.363
4,356,India,21206,Mumbai (Bombay),NaN,19.073975,72.880838,2857.359,3432.147,4060.373,4853.571,5811.304,7081.960,8657.886,10390.912,12436.423,14309.845,16366.787,17890.736,19421.983,21042.538,22838.483,25207.329,27796.555


In [28]:
# Rename country to normalized convention 

city_pop_df['test_country'] = city_pop_df['Country or area'].map(lambda val: retrieve_country(val))

city_pop_df.test_country.isna().value_counts()

False    1619
True       73
Name: test_country, dtype: int64

In [29]:
for index, val in enumerate(city_pop_df.test_country):
    if val is np.nan:
        print(index, city_pop_df['Country or area'][index])

22 Democratic Republic of the Congo
39 Iran (Islamic Republic of)
45 China, Hong Kong SAR
142 Iran (Islamic Republic of)
149 Venezuela (Bolivarian Republic of)
153 Dem. People's Republic of Korea
206 Venezuela (Bolivarian Republic of)
219 Bolivia (Plurinational State of)
227 Democratic Republic of the Congo
229 Democratic Republic of the Congo
252 Iran (Islamic Republic of)
257 Bolivia (Plurinational State of)
259 Iran (Islamic Republic of)
271 Venezuela (Bolivarian Republic of)
286 Iran (Islamic Republic of)
304 Iran (Islamic Republic of)
380 Bolivia (Plurinational State of)
389 Iran (Islamic Republic of)
392 Iran (Islamic Republic of)
407 Democratic Republic of the Congo
410 Venezuela (Bolivarian Republic of)
471 Democratic Republic of the Congo
473 Venezuela (Bolivarian Republic of)
562 Iran (Islamic Republic of)
607 Democratic Republic of the Congo
676 Iran (Islamic Republic of)
710 Venezuela (Bolivarian Republic of)
712 Iran (Islamic Republic of)
730 Venezuela (Bolivarian Republic

In [30]:
country_map = {'Democratic Republic of the Congo':'Congo, The Democratic Republic of the',
               'Iran (Islamic Republic of)':'Iran, Islamic Republic of',
              'China, Hong Kong SAR':'Hong Kong',
               'Venezuela (Bolivarian Republic of)':"Venezuela, Bolivarian Republic of",
              '''Dem. People's Republic of Korea''':"Korea, Democratic People's Republic of",
              'Bolivia (Plurinational State of)':'Bolivia, Plurinational State of',
              'China, Macao SAR':'Macao',
              'TFYR Macedonia':'North Macedonia'}

country_zip = list(zip(city_pop_df['Country or area'], city_pop_df.test_country))

for i, (CN, TC) in enumerate(country_zip):
    if TC is np.nan:
        city_pop_df.test_country[i] = country_map[CN]

city_pop_df.test_country.isna().value_counts()

False    1692
Name: test_country, dtype: int64

In [31]:
# City/lat/long stuff here


### 5. Data Source: World Bank

[Doctors](https://data.worldbank.org/indicator/SH.MED.PHYS.ZS) and [hospital beds per 1000 people/country](https://data.worldbank.org/indicator/SH.MED.BEDS.ZS)


In [32]:
doctors_df = pd.read_csv('../API_SH.MED.PHYS.ZS_DS2_en_csv_v2_713164/API_SH.MED.PHYS.ZS_DS2_en_csv_v2_713164.csv', 
                         skiprows=4)
doctors_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,0.035,NaN,NaN,NaN,NaN,0.063,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.077,NaN,NaN,NaN,NaN,0.183,0.179,NaN,0.129,0.109,NaN,NaN,0.143,NaN,NaN,NaN,0.1100,NaN,NaN,NaN,0.1957,NaN,NaN,NaN,NaN,0.163,0.1774,0.1771,0.2156,0.2396,0.2553,0.2450,0.2894,0.3039,0.2907,0.2840,NaN,NaN,NaN,NaN
2,Angola,AGO,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,0.067,NaN,NaN,NaN,NaN,0.076,NaN,NaN,NaN,NaN,0.116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.059,NaN,NaN,NaN,NaN,NaN,0.042,NaN,NaN,NaN,NaN,NaN,NaN,0.0584,NaN,NaN,NaN,NaN,NaN,NaN,0.0618,NaN,NaN,NaN,NaN,0.1311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2149,NaN,NaN,NaN
3,Albania,ALB,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,0.276,NaN,NaN,NaN,NaN,0.481,NaN,NaN,NaN,NaN,0.739,0.911,0.845,NaN,NaN,NaN,NaN,1.036,NaN,NaN,1.367,NaN,NaN,1.415,1.407,1.406,NaN,NaN,NaN,1.403,1.374,1.47,1.65,1.425,NaN,1.306,1.354,1.2950,1.289,1.282,1.389,NaN,1.305,NaN,NaN,NaN,NaN,1.1460,NaN,1.1440,1.2379,1.2225,1.2658,1.2706,NaN,NaN,1.1998,NaN,NaN,NaN,NaN
4,Andorra,AND,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.231,NaN,2.4350,2.470,2.594,2.549,2.5940,NaN,3.3333,NaN,NaN,3.640,3.7160,NaN,3.1120,4.0000,NaN,NaN,NaN,NaN,3.3333,NaN,NaN,NaN,NaN,NaN


In [33]:
doctors_df['latest_doctors_per_1000'] = doctors_df.ffill(axis=1).iloc[:, -1] 

for index, val in enumerate(doctors_df.latest_doctors_per_1000):
    if val == 'SH.MED.PHYS.ZS':
        doctors_df.latest_doctors_per_1000[index] = np.nan
    else:
        doctors_df.latest_doctors_per_1000[index] = float(val)

# impute median into 11 country entries
doctors_df.latest_doctors_per_1000.fillna(doctors_df.latest_doctors_per_1000.median(), inplace = True)

doctors_df.latest_doctors_per_1000.isna().value_counts()

False    264
Name: latest_doctors_per_1000, dtype: int64

In [34]:
hosp_beds_df = pd.read_csv('../API_SH.MED.BEDS.ZS_DS2_en_csv_v2_717551/API_SH.MED.BEDS.ZS_DS2_en_csv_v2_717551.csv', 
                           skiprows=4)
hosp_beds_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,0.170627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2756,NaN,NaN,NaN,NaN,NaN,0.3091,NaN,NaN,0.2498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,0.40,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.5,0.5,0.5,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,2.061462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,5.102676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.2717,4.1871,4.1607,4.0862,4.139,4.1388,4.0697,3.9700,3.9355,4.1321,4.0249,3.9987,4.0134,3.8314,3.0171,3.19,3.14,3.05,3.05,3.03,3.3,3.30,3.1,3.1,3.0,3.1,3.1,3.1,NaN,2.8,3.0,2.6,2.9,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.18,3.00,3.09,3.20,3.2,2.59,NaN,3.3,NaN,2.7,2.6,2.6,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
hosp_beds_df['latest_hosp_beds_per_1000'] = hosp_beds_df.ffill(axis=1).iloc[:, -1] 

for index, val in enumerate(hosp_beds_df.latest_hosp_beds_per_1000):
    if val == 'SH.MED.BEDS.ZS':
        hosp_beds_df.latest_hosp_beds_per_1000[index] = np.nan
    else:
        hosp_beds_df.latest_hosp_beds_per_1000[index] = float(val)

# impute median into 18 country entries
hosp_beds_df.latest_hosp_beds_per_1000.fillna(hosp_beds_df.latest_hosp_beds_per_1000.median(), inplace = True)

hosp_beds_df.latest_hosp_beds_per_1000.isna().value_counts()

False    264
Name: latest_hosp_beds_per_1000, dtype: int64

### 6. Begin merging country-level data

- SPAR index
- GDP per capita
- Physicians per 1000 people
- Hospital beds per 100 people

In [36]:
health_df = hosp_beds_df[['Country Name', 
                          'latest_hosp_beds_per_1000']].merge(doctors_df[['Country Name',
                                                                        'latest_doctors_per_1000']], 
                                                             how = 'inner', 
                                                             on = 'Country Name')

In [37]:
# Rename country to normalized convention 

health_df['test_country'] = health_df['Country Name'].map(lambda val: retrieve_country(val))

health_df.test_country.isna().value_counts()

False    196
True      68
Name: test_country, dtype: int64

In [38]:
for index, val in enumerate(health_df.test_country):
    if val is np.nan:
        print(index, health_df['Country Name'][index])

5 Arab World
21 Bahamas, The
34 Central Europe and the Baltics
36 Channel Islands
41 Congo, Dem. Rep.
42 Congo, Rep.
47 Caribbean small states
59 East Asia & Pacific (excluding high income)
60 Early-demographic dividend
61 East Asia & Pacific
62 Europe & Central Asia (excluding high income)
63 Europe & Central Asia
65 Egypt, Arab Rep.
66 Euro area
71 European Union
72 Fragile and conflict affected situations
77 Micronesia, Fed. Sts.
84 Gambia, The
93 High income
94 Hong Kong SAR, China
96 Heavily indebted poor countries (HIPC)
100 IBRD only
101 IDA & IBRD total
102 IDA total
103 IDA blend
105 IDA only
108 Not classified
110 Iran, Islamic Rep.
123 St. Kitts and Nevis
124 Korea, Rep.
126 Latin America & Caribbean (excluding high income)
127 Lao PDR
131 St. Lucia
132 Latin America & Caribbean
133 Least developed countries: UN classification
134 Low income
137 Lower middle income
138 Low & middle income
140 Late-demographic dividend
144 Macao SAR, China
145 St. Martin (French part)
151 Mid

In [39]:
health_df = health_df.drop(labels=[5, 34, 47, 59, 60, 61, 62, 63, 66, 71, 72, 93, 96, 100, 
                                   101, 102, 103, 105, 126, 132, 133, 134, 137, 138, 140, 151, 
                                   154, 159, 168, 179, 181, 189, 195, 196, 202, 213, 215, 216,
                                   228, 229, 234, 236, 238, 239, 247, 257], 
                           axis = 0).reset_index()

country_map = {'Congo, Dem. Rep.':'Congo, The Democratic Republic of the',
               'Congo, Rep.':'Congo',
              'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines',
               'Venezuela, RB':"Venezuela, Bolivarian Republic of",
              'West Bank and Gaza':"Palestine, State of",
              'Bolivia (Plurinational State of)':'Bolivia, Plurinational State of',
              'Virgin Islands (U.S.)':'Virgin Islands, U.S.',
              'Yemen, Rep.':'Yemen',
              'Korea, Dem. People’s Rep.':"Korea, Democratic People's Republic of",
              'St. Martin (French part)':'Saint Martin (French part)',
              'Macao SAR, China':'Macao',
              'St. Lucia':'Saint Lucia',
              'Lao PDR':"Lao People's Democratic Republic",
              'Korea, Rep.':'Korea, Republic of',
              'St. Kitts and Nevis':'Saint Kitts and Nevis',
              'Iran, Islamic Rep.':'Iran, Islamic Republic of',
              'Hong Kong SAR, China':'Hong Kong',
              'Gambia, The':'Gambia',
              'Micronesia, Fed. Sts.':'Micronesia, Federated States of',
              'Egypt, Arab Rep.':'Egypt',
              'Bahamas, The':'Bahamas', 
              'Not classified': 'Others',
              'Channel Islands':'Channel Islands'}

country_zip = list(zip(health_df['Country Name'], health_df.test_country))

for i, (CN, TC) in enumerate(country_zip):
    if TC is np.nan:
        health_df.test_country[i] = country_map[CN]

health_df.test_country.isna().value_counts()

False    218
Name: test_country, dtype: int64

In [40]:
gdp_df.rename(columns={'test_country': 'test_country', 'Value': 'GDP_per_capita'}, inplace = True)

country_stats_df = health_df.merge(gdp_df[['test_country', 'GDP_per_capita']], how = 'outer', on = 'test_country')

spar_df2.rename(columns = {'test_country': 'test_country', 'Index': 'SPAR_index'}, inplace = True)
country_stats_df = country_stats_df.merge(spar_df2, how = 'outer', on = 'test_country')

# impute median SPAR index for 112 country entries
country_stats_df.SPAR_index.fillna(country_stats_df.SPAR_index.median(), inplace = True)

# impute median GDP into 51 country entries
country_stats_df.GDP_per_capita.fillna(country_stats_df.GDP_per_capita.median(), inplace = True)

# impute median hosp beds and physicians into 17 country entries
country_stats_df.latest_hosp_beds_per_1000.fillna(country_stats_df.latest_hosp_beds_per_1000.median(), inplace = True)
country_stats_df.latest_doctors_per_1000.fillna(country_stats_df.latest_doctors_per_1000.median(), inplace = True)

country_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231 entries, 0 to 230
Data columns (total 8 columns):
index                        227 non-null float64
Country Name                 227 non-null object
latest_hosp_beds_per_1000    231 non-null float64
latest_doctors_per_1000      231 non-null float64
test_country                 231 non-null object
GDP_per_capita               231 non-null float64
Country                      204 non-null object
SPAR_index                   231 non-null float64
dtypes: float64(5), object(3)
memory usage: 16.2+ KB


### 7. Data Source: [Rand Corporation](https://www.rand.org/pubs/research_reports/RR1605.html)

Infectious Disease Vulnerability Index (IDVI) takes data from various international organizations to determine scores per each of seven domains as well as an overall score. The domain categories are:
- Demographic
- Health Care
- Public Health
- Disease Dynamics
- Political-Domestic
- Political-International
- Economic

In [41]:
IVDI_df = pd.read_csv('../Rand_IVDF.csv', sep = ' ', quotechar = "'")
IVDI_df.head()

,Rank,Country or Territory,Combatant Command Overall Score,Normed Demographic Domain Score,Health Care Domain Score,Public Health Domain Score,Disease Dynamics Domain Score,Political- Domestic Domain Score,Political- International Domain Score,Economic Domain Score
1,Somalia,AFRICOM,0.000000,0.348761,0.058301,0.000000,0.629775,0.000000,0.102937,0.075656
2,Central African Republic,AFRICOM,0.000061,0.155293,0.000000,0.045047,0.548853,0.053251,0.315859,0.005424
3,Chad,AFRICOM,0.098450,0.210246,0.049051,0.188812,0.543405,0.115587,0.259991,0.106760
4,South Sudan,AFRICOM,0.100836,0.007305,0.231602,0.189689,0.415040,0.084988,0.355877,0.186605
5,Mauritania,AFRICOM,0.107294,0.303530,0.200198,0.013730,0.542587,0.199660,0.267845,0.219330


In [42]:
cols_to_avg = list(IVDI_df.columns)
cols_to_avg.remove('Rank')
cols_to_avg.remove('Country or Territory')
cols_to_avg

['Combatant Command Overall Score',
 'Normed Demographic Domain Score',
 'Health Care Domain Score',
 'Public Health Domain Score',
 'Disease Dynamics Domain Score',
 'Political- Domestic Domain Score',
 'Political- International Domain Score',
 'Economic Domain Score']

In [43]:
IVDI_df[cols_to_avg] = IVDI_df[cols_to_avg].astype(float)

In [44]:
IVDI_df['Overall Score'] = 0
IVDI_df['Overall Score'] = IVDI_df[cols_to_avg].mean(axis=1)

IVDI_df.head()

,Rank,Country or Territory,Combatant Command Overall Score,Normed Demographic Domain Score,Health Care Domain Score,Public Health Domain Score,Disease Dynamics Domain Score,Political- Domestic Domain Score,Political- International Domain Score,Economic Domain Score,Overall Score
1,Somalia,AFRICOM,0.000000,0.348761,0.058301,0.000000,0.629775,0.000000,0.102937,0.075656,0.151929
2,Central African Republic,AFRICOM,0.000061,0.155293,0.000000,0.045047,0.548853,0.053251,0.315859,0.005424,0.140473
3,Chad,AFRICOM,0.098450,0.210246,0.049051,0.188812,0.543405,0.115587,0.259991,0.106760,0.196538
4,South Sudan,AFRICOM,0.100836,0.007305,0.231602,0.189689,0.415040,0.084988,0.355877,0.186605,0.196493
5,Mauritania,AFRICOM,0.107294,0.303530,0.200198,0.013730,0.542587,0.199660,0.267845,0.219330,0.231772


In [45]:
IVDI_df = IVDI_df[['Rank', 'Overall Score']]
IVDI_df.rename(columns = {'Rank': 'Country Name', 'Overall Score': 'IVDI_score'}, inplace = True)
IVDI_df.head()

,Country Name,IVDI_score
1,Somalia,0.151929
2,Central African Republic,0.140473
3,Chad,0.196538
4,South Sudan,0.196493
5,Mauritania,0.231772


In [46]:
IVDI_df['test_country'] = IVDI_df['Country Name'].map(lambda val: retrieve_country(val))

IVDI_df.test_country.isna().value_counts()

False    187
True       8
Name: test_country, dtype: int64

In [47]:
IVDI_df = IVDI_df.reset_index()

In [48]:
for index, val in enumerate(IVDI_df.test_country):
    if val is np.nan:
        print(index, IVDI_df['Country Name'][index])

10 Democratic Republic of the Congo
25 Republic of the Congo (CongoBrazzaville)
27 Côte d’Ivoire
39 Laos
42 Swaziland
45 Democratic People’s Republic of Korea (North Korea)
64 Cape Verde
178 Republic of Korea (South Korea)


In [49]:
country_map = {'Democratic Republic of the Congo':'Congo, The Democratic Republic of the',
               'Republic of the Congo (CongoBrazzaville)':'Congo',
              'Côte d’Ivoire':"Côte d'Ivoire",
               'Laos':"Lao People's Democratic Republic",
              'Swaziland':'Eswatini',
              'Democratic People’s Republic of Korea (North Korea)':"Korea, Democratic People's Republic of",
              'Cape Verde':'Cabo Verde',
              'Republic of Korea (South Korea)':'Korea, Republic of'}

country_zip = list(zip(IVDI_df['Country Name'], IVDI_df.test_country))

for i, (CN, TC) in enumerate(country_zip):
    if TC is np.nan:
        IVDI_df.test_country[i] = country_map[CN]

IVDI_df.test_country.isna().value_counts()

False    195
Name: test_country, dtype: int64

### 8. Continue merging country-level data

- country_stats
    - SPAR index
    - GDP per capita
    - Physicians per 1000 people
    - Hospital beds per 100 people
- IVDI

In [50]:
country_df = country_stats_df = country_stats_df.merge(IVDI_df[['test_country', 'IVDI_score']], how = 'outer', on = 'test_country')
country_df.drop(columns=['Country Name', 'Country', 'index'], inplace=True)
country_df.head()

,latest_hosp_beds_per_1000,latest_doctors_per_1000,test_country,GDP_per_capita,SPAR_index,IVDI_score
0,2.400337,1.1200,Aruba,13536.794829,60.463238,NaN
1,0.500000,0.2840,Afghanistan,1955.006208,34.000000,0.257002
2,0.800000,0.2149,Angola,6452.355165,58.769231,0.279402
3,2.900000,1.1998,Albania,13364.155397,60.463238,0.636599
4,2.500000,3.3333,Andorra,13536.794829,36.692308,0.679377


In [51]:
country_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 243
Data columns (total 6 columns):
latest_hosp_beds_per_1000    243 non-null float64
latest_doctors_per_1000      243 non-null float64
test_country                 244 non-null object
GDP_per_capita               243 non-null float64
SPAR_index                   243 non-null float64
IVDI_score                   217 non-null float64
dtypes: float64(5), object(1)
memory usage: 13.3+ KB


### 9. Look at latitude/longitude using geopy to normalize locations

- City population densities
- JHU: provinces/states with confirmed coronavirus cases
- ROCS: airport nodes

In [52]:
city_pop_df = city_pop_df[['test_country','Urban Agglomeration', 
                           'Latitude', 'Longitude', '2020']]
city_pop_df.head()

,test_country,Urban Agglomeration,Latitude,Longitude,2020
0,Japan,Tokyo,35.689500,139.691710,38323.229
1,India,Delhi,28.666670,77.216670,29347.622
2,China,Shanghai,31.222220,121.458060,27137.316
3,Brazil,São Paulo,-23.547500,-46.636110,22118.900
4,India,Mumbai (Bombay),19.073975,72.880838,22838.483


In [53]:
rocs_df2.drop(columns = 'country_name', inplace = True)
rocs_df2.head()

,effective_dist,geo_dist,airport_name,continent_name,region_name,airport_id,ann_pass_flux,connected_airports,long,lat,test_country
0,9.626587,8111.034829,Addis Ababa,Africa,Eastern Africa,ADD,3887132.5,56.5,38.799444,8.977778,Ethiopia
1,13.897219,7763.569189,Asmara,Africa,Eastern Africa,ASM,307820.5,10.5,38.910556,15.291944,Eritrea
2,14.024414,9712.495427,Bujumbura,Africa,Eastern Africa,BJM,252165.0,6.5,29.318611,-3.323889,Burundi
3,13.695018,8990.314878,Dar es Salaam,Africa,Eastern Africa,DAR,1840029.0,17.0,39.202500,-6.878056,Tanzania
4,13.038729,9221.694455,Entebbe,Africa,Eastern Africa,EBB,1079933.0,16.5,32.443611,0.042500,Uganda


In [37]:
jhu_df.drop(columns = 'Country/Region', inplace = True)
jhu_df.head()

,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,country
0,Anhui,31.8257,117.2264,1,9,15,39,60,70,106,152,200,237,297,340,408,480,530,591,665,733,779,830,860,889,910,934,950,962,973,982,986,987,988,989,989,989,989,989,989,China
1,Beijing,40.1824,116.4142,14,22,36,41,68,80,91,111,114,139,168,191,212,228,253,274,297,315,326,337,342,352,366,372,375,380,381,387,393,395,396,399,399,399,400,400,410,China
2,Chongqing,30.0572,107.8740,6,9,27,57,75,110,132,147,182,211,247,300,337,366,389,411,426,428,468,486,505,518,529,537,544,551,553,555,560,567,572,573,575,576,576,576,576,China
3,Fujian,26.0789,117.9874,1,5,10,18,35,59,80,84,101,120,144,159,179,194,205,215,224,239,250,261,267,272,279,281,285,287,290,292,293,293,293,293,293,293,294,294,296,China
4,Gansu,36.0611,103.8343,0,2,2,4,7,14,19,24,26,29,40,51,55,57,62,62,67,79,83,83,86,87,90,90,90,90,91,91,91,91,91,91,91,91,91,91,91,China


In [38]:
geolocator = Nominatim(user_agent="apple.chrome", timeout=10)
rgeocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [39]:
jhu_df['lat_long'] = jhu_df.Lat.map(str) + ',' + jhu_df.Long.map(str)
jhu_df.head()

,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,country,lat_long
0,Anhui,31.8257,117.2264,1,9,15,39,60,70,106,152,200,237,297,340,408,480,530,591,665,733,779,830,860,889,910,934,950,962,973,982,986,987,988,989,989,989,989,989,989,China,"31.8257,117.2264"
1,Beijing,40.1824,116.4142,14,22,36,41,68,80,91,111,114,139,168,191,212,228,253,274,297,315,326,337,342,352,366,372,375,380,381,387,393,395,396,399,399,399,400,400,410,China,"40.1824,116.4142"
2,Chongqing,30.0572,107.8740,6,9,27,57,75,110,132,147,182,211,247,300,337,366,389,411,426,428,468,486,505,518,529,537,544,551,553,555,560,567,572,573,575,576,576,576,576,China,"30.0572,107.874"
3,Fujian,26.0789,117.9874,1,5,10,18,35,59,80,84,101,120,144,159,179,194,205,215,224,239,250,261,267,272,279,281,285,287,290,292,293,293,293,293,293,293,294,294,296,China,"26.0789,117.9874"
4,Gansu,36.0611,103.8343,0,2,2,4,7,14,19,24,26,29,40,51,55,57,62,62,67,79,83,83,86,87,90,90,90,90,91,91,91,91,91,91,91,91,91,91,91,China,"36.0611,103.8343"


In [40]:
def city_county_state(coord):
    location = rgeocode(coord, language = 'en')
    try:
        county = location.raw['address']['county']
    except:
        county = ''
    try:
        state = location.raw['address']['state']
    except:
        state = ''
    try:     
        city = location.raw['address']['city']
    except:   
        city = ''
    return f'{city}, {county}, {state}'

In [41]:
jhu_df['city_county_state'] = jhu_df['lat_long'].apply(city_county_state)
                                                   
jhu_df.head()

,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,country,lat_long,city_county_state
0,Anhui,31.8257,117.2264,1,9,15,39,60,70,106,152,200,237,297,340,408,480,530,591,665,733,779,830,860,889,910,934,950,962,973,982,986,987,988,989,989,989,989,989,989,China,"31.8257,117.2264","Lianhua Community, 合肥政务文化新区, Anhui"
1,Beijing,40.1824,116.4142,14,22,36,41,68,80,91,111,114,139,168,191,212,228,253,274,297,315,326,337,342,352,366,372,375,380,381,387,393,395,396,399,399,399,400,400,410,China,"40.1824,116.4142",", Changping District, Beijing"
2,Chongqing,30.0572,107.8740,6,9,27,57,75,110,132,147,182,211,247,300,337,366,389,411,426,428,468,486,505,518,529,537,544,551,553,555,560,567,572,573,575,576,576,576,576,China,"30.0572,107.874",", Fengdu County, Chongqing"
3,Fujian,26.0789,117.9874,1,5,10,18,35,59,80,84,101,120,144,159,179,194,205,215,224,239,250,261,267,272,279,281,285,287,290,292,293,293,293,293,293,293,294,294,296,China,"26.0789,117.9874","Sanming City, Youxi County, Fujian"
4,Gansu,36.0611,103.8343,0,2,2,4,7,14,19,24,26,29,40,51,55,57,62,62,67,79,83,83,86,87,90,90,90,90,91,91,91,91,91,91,91,91,91,91,91,China,"36.0611,103.8343","Guangwumen Subdistrict, Chengguan District, Gansu"


In [42]:
jhu_df.city_county_state.value_counts()

Yokohama, , Kanagawa Prefecture                                   3
, Sacramento County, California                                   1
, Viken,                                                          1
北辰区, Beichen District, Tianjin                                    1
, Hòa Vang District, Da Nang                                      1
Chuxiong City, , Yunnan                                           1
Tempe, Maricopa County, Arizona                                   1
Madison, Dane County, Wisconsin                                   1
, Yongji County, Jilin                                            1
Isaneshwor, Madhya Nepal, Gandaki Pradesh                         1
, , Obwalden                                                      1
, , Phnom Penh                                                    1
Ka'erquga, Yuli County, Xinjiang                                  1
Los Angeles, Los Angeles County, California                       1
Chicago, Cook County, Illinois                  

In [66]:
# Repeat steps for ROCS and city population datasets

rocs_df2['lat_long'] = rocs_df2.lat.map(str) + ',' + rocs_df2.long.map(str)
rocs_df2['city_county_state'] = rocs_df2['lat_long'].apply(city_county_state)

In [72]:
city_pop_df['lat_long'] = city_pop_df.Latitude.map(str) + ',' + city_pop_df.Longitude.map(str)
city_pop_df['city_county_state'] = city_pop_df['lat_long'].apply(city_county_state)

In [44]:
print('JHU: ', len(jhu_df), len(jhu_df['city_county_state'].unique()))

jhu_df.loc[jhu_df['city_county_state'] == 'Yokohama, , Kanagawa Prefecture']

JHU:  105 103


,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,country,lat_long,city_county_state
71,Diamond Princess cruise ship,35.4437,139.638,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,61,64,135,135,175,175,218,285,355,454,542,621,634,634,634,691,691,691,705,705,Diamond Princess cruise ship,"35.4437,139.638","Yokohama, , Kanagawa Prefecture"
78,From Diamond Princess,35.4437,139.638,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,7,7,7,7,7,8,Australia,"35.4437,139.638","Yokohama, , Kanagawa Prefecture"
84,Unassigned Location (From Diamond Princess),35.4437,139.638,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,36,42,42,United States,"35.4437,139.638","Yokohama, , Kanagawa Prefecture"


In [45]:
jhu_df.to_csv('jhu_df_for_imputation.csv')
# rocs_df2.to_csv('rocs_df2.csv')
# city_pop_df.to_csv('city_pop_df.csv')
# country_df.to_csv('country_df.csv')

In [ ]:
# Princess Diamond Cruise ship (country = Others) docked in Australia, where it spread. 
# Also spread to Omaha, NE.

print('ROCS: ', len(rocs_df2), len(rocs_df2['city_county_state'].unique()))
print('City pop: ', len(city_pop_df), len(city_pop_df['city_county_state'].unique()))

In [16]:
jhu_df = pd.read_csv('jhu_df_imputed.csv', index_col = 0)

In [25]:
jhu_df.head()

,1/10/20,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,12/08/19,12/11/19,12/12/19,12/15/19,12/17/19,12/18/19,12/19/19,12/20/19,12/21/19,12/22/19,12/23/19,12/24/19,12/25/19,12/26/19,12/27/19,12/28/29,12/29/19,12/30/19,12/31/19,2/1/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/2/20,2/20/20,2/21/20,2/22/20,2/23/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,Lat,Long,Province/State,city_county_state,lat_long,test_country
0,1,1.0,9.0,15.0,39.0,60.0,70.0,106.0,152.0,200.0,237.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,297.0,830.0,860.0,889.0,910.0,934.0,950.0,962.0,973.0,982.0,986.0,340.0,987.0,988.0,989.0,989.0,408.0,480.0,530.0,591.0,665.0,733.0,779.0,31.82571,117.2264,Anhui,"Lianhua Community, 合肥政务文化新区, Anhui","31.825709999999997,117.2264",China
1,1,14.0,22.0,36.0,41.0,68.0,80.0,91.0,111.0,114.0,139.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,168.0,337.0,342.0,352.0,366.0,372.0,375.0,380.0,381.0,387.0,393.0,191.0,395.0,396.0,399.0,399.0,212.0,228.0,253.0,274.0,297.0,315.0,326.0,40.18238,116.4142,Beijing,", Changping District, Beijing","40.18238,116.4142",China
2,1,6.0,9.0,27.0,57.0,75.0,110.0,132.0,147.0,182.0,211.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,247.0,486.0,505.0,518.0,529.0,537.0,544.0,551.0,553.0,555.0,560.0,300.0,567.0,572.0,573.0,575.0,337.0,366.0,389.0,411.0,426.0,428.0,468.0,30.05718,107.8740,Chongqing,", Fengdu County, Chongqing","30.05718,107.874",China
3,1,1.0,5.0,10.0,18.0,35.0,59.0,80.0,84.0,101.0,120.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144.0,261.0,267.0,272.0,279.0,281.0,285.0,287.0,290.0,292.0,293.0,159.0,293.0,293.0,293.0,293.0,179.0,194.0,205.0,215.0,224.0,239.0,250.0,26.07783,117.9895,Fujian,"Sanming City, Youxi County, Fujian","26.07783,117.9895",China
4,1,0.0,2.0,2.0,4.0,7.0,14.0,19.0,24.0,26.0,29.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40.0,83.0,86.0,87.0,90.0,90.0,90.0,90.0,91.0,91.0,91.0,51.0,91.0,91.0,91.0,91.0,55.0,57.0,62.0,62.0,67.0,79.0,83.0,36.06110,103.8343,Gansu,"Guangwumen Subdistrict, Chengguan District, Gansu","36.0611,103.8343",China


In [21]:
jhu_df.groupby('test_country').sum()

,1/10/20,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,12/08/19,12/11/19,12/12/19,12/15/19,12/17/19,12/18/19,12/19/19,12/20/19,12/21/19,12/22/19,12/23/19,12/24/19,12/25/19,12/26/19,12/27/19,12/28/29,12/29/19,12/30/19,12/31/19,2/1/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/2/20,2/20/20,2/21/20,2/22/20,2/23/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,Lat,Long
test_country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,0,0.0,0.0,0.0,0.0,4.0,5.0,5.0,6.0,9.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,12.0,15.0,19.0,22.0,22.0,12.0,13.0,13.0,14.0,15.0,15.0,15.0,-98.63700,727.43620
Belgium,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,50.50390,4.46990
Cambodia,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,12.56570,104.99100
Canada,0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,8.0,8.0,8.0,4.0,8.0,9.0,9.0,9.0,4.0,4.0,5.0,5.0,7.0,7.0,7.0,135.92080,-283.74950
China,19,547.0,639.0,916.0,1399.0,2062.0,2863.0,5494.0,6070.0,8124.0,9783.0,1,2,5,8,12,14,16,25,29,37,40,45,47,49,59,68,78,90,104,11871.0,42306.0,44327.0,44699.0,59832.0,66292.0,68347.0,70446.0,72364.0,74139.0,74546.0,16607.0,74999.0,75472.0,76922.0,76938.0,19693.0,23680.0,27409.0,30553.0,34075.0,36778.0,39790.0,1033.98047,3458.52786
Egypt,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.82060,30.80250
Finland,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,61.92410,25.74820
France,0,0.0,0.0,2.0,3.0,3.0,3.0,4.0,5.0,5.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,11.0,11.0,11.0,11.0,11.0,12.0,12.0,12.0,12.0,12.0,6.0,12.0,12.0,12.0,12.0,6.0,6.0,6.0,6.0,6.0,11.0,11.0,46.22760,2.21370
Germany,0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,4.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,14.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,10.0,16.0,16.0,16.0,16.0,12.0,12.0,12.0,12.0,13.0,13.0,14.0,51.16570,10.45150


In [26]:
jhu_df.test_country.value_counts()

China                                     31
United States                             16
Australia                                  5
Canada                                     3
Spain                                      1
Hong Kong                                  1
Thailand                                   1
India                                      1
Iran, Islamic Republic of                  1
Russian Federation                         1
Finland                                    1
Others                                     1
Viet Nam                                   1
Taiwan, Province of China                  1
France                                     1
Egypt                                      1
Philippines                                1
Iraq                                       1
Lebanon                                    1
Sweden                                     1
Nepal                                      1
Belgium                                    1
Malaysia  

In [27]:
jhu_df.city_county_state.value_counts()

Yokohama, , Kanagawa Prefecture                                                     2
Madrid, Área metropolitana de Madrid y Corredor del Henares, Community of Madrid    1
, Samudrapur, Maharashtra                                                           1
, Jung-gu,                                                                          1
, Gila County, Arizona                                                              1
Rome, Roma Capitale, Lazio                                                          1
Vancouver, Metro Vancouver Regional District, British Columbia                      1
, Tazewell County, Illinois                                                         1
Omaha, Douglas County, Nebraska                                                     1
Zahlé, , Zahlé District                                                             1
, Ardekan County, Yazd Province                                                     1
Toronto, , Ontario                                    

In [8]:
jhu_gdf = gpd.read_file('jhu_df_imputed.csv')

In [9]:
jhu_gdf.head()

,field_1,1/10/20,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,12/08/19,12/11/19,12/12/19,12/15/19,12/17/19,12/18/19,12/19/19,12/20/19,12/21/19,12/22/19,12/23/19,12/24/19,12/25/19,12/26/19,12/27/19,12/28/29,12/29/19,12/30/19,12/31/19,2/1/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/2/20,2/20/20,2/21/20,2/22/20,2/23/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,Lat,Long,Province/State,city_county_state,lat_long,test_country,geometry
0,0,1,1.0,9.0,15.0,39.0,60.0,70.0,106.0,152.0,200.0,237.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,297.0,830.0,860.0,889.0,910.0,934.0,950.0,962.0,973.0,982.0,986.0,340.0,987.0,988.0,989.0,989.0,408.0,480.0,530.0,591.0,665.0,733.0,779.0,31.82571,117.2264,Anhui,"Lianhua Community, 合肥政务文化新区, Anhui","31.825709999999997,117.2264",China,None
1,1,1,14.0,22.0,36.0,41.0,68.0,80.0,91.0,111.0,114.0,139.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,168.0,337.0,342.0,352.0,366.0,372.0,375.0,380.0,381.0,387.0,393.0,191.0,395.0,396.0,399.0,399.0,212.0,228.0,253.0,274.0,297.0,315.0,326.0,40.18238,116.4142,Beijing,", Changping District, Beijing","40.18238,116.4142",China,None
2,2,1,6.0,9.0,27.0,57.0,75.0,110.0,132.0,147.0,182.0,211.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,247.0,486.0,505.0,518.0,529.0,537.0,544.0,551.0,553.0,555.0,560.0,300.0,567.0,572.0,573.0,575.0,337.0,366.0,389.0,411.0,426.0,428.0,468.0,30.05718,107.874,Chongqing,", Fengdu County, Chongqing","30.05718,107.874",China,None
3,3,1,1.0,5.0,10.0,18.0,35.0,59.0,80.0,84.0,101.0,120.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144.0,261.0,267.0,272.0,279.0,281.0,285.0,287.0,290.0,292.0,293.0,159.0,293.0,293.0,293.0,293.0,179.0,194.0,205.0,215.0,224.0,239.0,250.0,26.07783,117.9895,Fujian,"Sanming City, Youxi County, Fujian","26.07783,117.9895",China,None
4,4,1,0.0,2.0,2.0,4.0,7.0,14.0,19.0,24.0,26.0,29.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40.0,83.0,86.0,87.0,90.0,90.0,90.0,90.0,91.0,91.0,91.0,51.0,91.0,91.0,91.0,91.0,55.0,57.0,62.0,62.0,67.0,79.0,83.0,36.0611,103.8343,Gansu,"Guangwumen Subdistrict, Chengguan District, Gansu","36.0611,103.8343",China,None


In [15]:
points = gpd.GeoDataFrame(jhu_gdf, geometry = gpd.points_from_xy(jhu_gdf.Long, jhu_gdf.Lat))

AttributeError: module 'geopandas' has no attribute 'points_from_xy'

### 10. Merge all datasets by location

- All country stats (GDP, physicians, hospitals, IVDI, SPAR index)
- City population densities
- JHU: provinces/states with confirmed coronavirus cases (merge locations only)
- ROCS: airport nodes

In [151]:
rocs_df2 = pd.read_csv('rocs_df2.csv', index_col = 0)
city_pop_df = pd.read_csv('city_pop_df.csv', index_col = 0)
country_df = pd.read_csv('country_df.csv', index_col = 0)

In [152]:
rocs_df2['tmp'] = 1
city_pop_df['tmp'] = 1

In [153]:
# merge city_pop_df, rocs_df2 on (city_county_state, country) -- then jhu_df locations
city_df = pd.merge(city_pop_df, rocs_df2, on = 'tmp')
city_df = city_df.rename(columns = 
                         {'2020':
                          'city_pop'}).drop(columns = 
                                                   ['Urban Agglomeration' 
#                                                     'Latitude',
#                                                     'Longitude', 
#                                                     'lat_long_x'])
                                                   ])

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [154]:
city_df['dist'] = city_df.apply(lambda row: haversine(row['Longitude'], 
                                            row['Latitude'], 
                                            row['long'], 
                                            row['lat']), axis=1)

In [155]:
city_df_500 = city_df[city_df.dist < 500]

print(len(city_df), len(city_df_500), len(city_df) - len(city_df_500))

2186064 22269 2163795


In [156]:
city_df_500 = city_df_500[~(city_df_500['effective_dist'].isna())]

print(len(city_df_500), len(city_pop_df), len(rocs_df2))

22269 1692 1292


In [157]:
city_df_500 = city_df_500.groupby('airport_name').mean()

city_df_500.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1214 entries, A Coruna to Zurich Airport
Data columns (total 11 columns):
Latitude              1214 non-null float64
Longitude             1214 non-null float64
city_pop              1214 non-null float64
tmp                   1214 non-null int64
effective_dist        1214 non-null float64
geo_dist              1214 non-null float64
ann_pass_flux         1214 non-null float64
connected_airports    1214 non-null float64
long                  1214 non-null float64
lat                   1214 non-null float64
dist                  1214 non-null float64
dtypes: float64(10), int64(1)
memory usage: 113.8+ KB


In [158]:
city_df_500.drop(columns = ['Latitude', 'Longitude', 'dist'], inplace = True)

In [160]:
city_df_500.reset_index(inplace = True)
city_df_500.head()

,airport_name,city_pop,tmp,effective_dist,geo_dist,ann_pass_flux,connected_airports,long,lat
0,A Coruna,667.350667,1,13.823357,9919.181663,593846.0,10.0,-8.377222,43.302222
1,Aalborg Airport,1088.171400,1,12.510090,7975.305048,948024.0,15.5,9.849167,57.092778
2,Aarhus Tirstrup Airport,1149.098222,1,12.860097,7975.995149,318795.5,14.0,10.619444,56.303889
3,Abbotsford,1799.046600,1,14.307867,9525.273098,312470.0,4.0,-122.360556,49.025278
4,Aberdeen,837.710467,1,11.998999,8575.481965,1884939.0,28.5,-2.200278,57.204167


In [198]:
jhu_df['tmp'] = 1

total_geo_df = pd.merge(city_df_500, jhu_df, on = 'tmp')

In [199]:
total_geo_df['dist'] = total_geo_df.apply(lambda row: haversine(row['Long'], 
                                            row['Lat'], 
                                            row['long'], 
                                            row['lat']), axis=1)

In [200]:
total_geo_df_500 = total_geo_df[total_geo_df.dist < 500]

print(len(total_geo_df), len(total_geo_df_500), len(total_geo_df) - len(total_geo_df_500))

127470 1518 125952


In [201]:
total_geo_df_500.head()

,airport_name,city_pop,tmp,effective_dist,geo_dist,ann_pass_flux,connected_airports,long,lat,1/10/20,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,12/08/19,12/11/19,12/12/19,12/15/19,12/17/19,12/18/19,12/19/19,12/20/19,12/21/19,12/22/19,12/23/19,12/24/19,12/25/19,12/26/19,12/27/19,12/28/29,12/29/19,12/30/19,12/31/19,2/1/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/2/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,Lat,Long,Province/State,city_county_state,country,lat_long,dist
204,Aalborg Airport,1088.171400,1,12.510090,7975.305048,948024.0,15.5,9.849167,57.092778,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.4720,8.4689,0,", Viken,",Norway,"60.472,8.4689",384.057831
206,Aalborg Airport,1088.171400,1,12.510090,7975.305048,948024.0,15.5,9.849167,57.092778,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.2639,9.5018,0,", Silkeborg Municipality, Central Denmark Region",Denmark,"56.2639,9.5018",94.577591
309,Aarhus Tirstrup Airport,1149.098222,1,12.860097,7975.995149,318795.5,14.0,10.619444,56.303889,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.4720,8.4689,0,", Viken,",Norway,"60.472,8.4689",480.057122
311,Aarhus Tirstrup Airport,1149.098222,1,12.860097,7975.995149,318795.5,14.0,10.619444,56.303889,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.2639,9.5018,0,", Silkeborg Municipality, Central Denmark Region",Denmark,"56.2639,9.5018",69.125492
350,Abbotsford,1799.046600,1,14.307867,9525.273098,312470.0,4.0,-122.360556,49.025278,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,47.6062,-122.3321,"Seattle, WA","Seattle, King County, Washington",United States,"47.6062,-122.3321",157.808301


In [203]:
total_geo_df_500.columns

Index(['airport_name', 'city_pop', 'tmp', 'effective_dist', 'geo_dist', 'ann_pass_flux', 'connected_airports', 'long', 'lat', '1/10/20', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '12/08/19', '12/11/19', '12/12/19', '12/15/19', '12/17/19', '12/18/19', '12/19/19', '12/20/19', '12/21/19', '12/22/19', '12/23/19', '12/24/19', '12/25/19', '12/26/19', '12/27/19', '12/28/29', '12/29/19', '12/30/19', '12/31/19', '2/1/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/2/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', 'Lat', 'Long', 'Province/State', 'city_county_state', 'country', 'lat_long', 'dist'], dtype='object')

In [204]:
total_geo_df_500.drop(columns = ['tmp','dist', 'lat_long','Lat','Long','long','lat'], inplace=True)

In [205]:
total_geo_df_500 = total_geo_df_500.groupby(['Province/State', 
                                             'city_county_state',
                                             'country']).mean().reset_index()


In [206]:
total_geo_df_500['12/08/19'].value_counts()

0    103
1      1
Name: 12/08/19, dtype: int64

In [219]:
# merge with country data and impute country-level data into city/airport entries
total_df = pd.merge(country_df.rename(columns = {'test_country':'country'}), 
                    total_geo_df_500, 
                    how = 'outer', 
                    on = 'country')

In [220]:
total_df = total_df.groupby(['Province/State', 'city_county_state','country']).mean().reset_index()

In [221]:
# Come back and impute airport information!!! we lose too much from the JHU dataset

# impute median into hospital beds, doctors, GDP, SPAR, IVDI
cols_to_imp = ['SPAR_index', 'GDP_per_capita', 'latest_hosp_beds_per_1000', 'latest_doctors_per_1000', 'IVDI_score']

for col in cols_to_imp:
    total_df[col].fillna(total_df[col].median(), inplace = True)

# add zeros to date columns, add all missing dates

new_dates = ['12/09/19', '12/10/19', '12/13/19', '12/14/19', '12/16/19', '01/01/20', '01/02/20',
            '01/03/20', '01/04/20', '01/05/20', '01/06/20', '01/07/20', '01/08/20', '01/09/20', 
             '01/11/20', '01/12/20', '01/13/20', '01/14/20','01/15/20', '01/16/20', '01/17/20',
            '01/18/20', '01/19/20', '01/20/20', '01/21/20']

total_df = pd.concat([total_df, pd.DataFrame(columns=new_dates)], axis=1)

total_df = total_df.fillna(0).rename(columns = {'12/28/29' : '12/28/19'})

In [222]:
total_df.drop_duplicates(inplace = True)

In [223]:
total_df.head()

,Province/State,city_county_state,country,latest_hosp_beds_per_1000,latest_doctors_per_1000,GDP_per_capita,SPAR_index,IVDI_score,city_pop,effective_dist,geo_dist,ann_pass_flux,connected_airports,1/10/20,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,12/08/19,12/11/19,12/12/19,12/15/19,12/17/19,12/18/19,12/19/19,12/20/19,12/21/19,12/22/19,12/23/19,12/24/19,12/25/19,12/26/19,12/27/19,12/28/19,12/29/19,12/30/19,12/31/19,2/1/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/2/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,12/09/19,12/10/19,12/13/19,12/14/19,12/16/19,01/01/20,01/02/20,01/03/20,01/04/20,01/05/20,01/06/20,01/07/20,01/08/20,01/09/20,01/11/20,01/12/20,01/13/20,01/14/20,01/15/20,01/16/20,01/17/20,01/18/20,01/19/20,01/20/20,01/21/20
0,0,", ,",San Marino,3.8,6.1468,13536.794829,32.846154,0.696439,811.059130,12.909125,8530.463669,3.312232e+06,54.719512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,", , Abu Dhabi",United Arab Emirates,1.2,2.3944,75075.257411,94.615385,0.718802,1210.169908,12.417796,5798.434510,9.665304e+06,70.277778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,8.0,8.0,8.0,8.0,8.0,8.0,9.0,9.0,9.0,9.0,5.0,9.0,9.0,13.0,13.0,13.0,13.0,13.0,13.0,5.0,5.0,5.0,5.0,5.0,7.0,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,", , Al Wusta",Oman,1.6,1.9697,41859.934477,100.000000,0.617149,1085.491700,11.108324,5690.297303,8.373158e+06,72.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,", , Jahra",Kuwait,2.0,2.5789,72897.562345,55.615385,0.668414,1290.116059,13.896034,6180.024430,2.496246e+06,28.187500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,26.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,", , Lopburi Province",Thailand,2.1,0.8096,19051.333382,79.461538,0.687330,1862.299400,7.352859,2074.761769,6.238800e+06,32.937500,0.0,2.0,3.0,5.0,7.0,8.0,8.0,14.0,14.0,14.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,32.0,33.0,33.0,33.0,33.0,33.0,34.0,35.0,35.0,35.0,19.0,35.0,35.0,35.0,35.0,35.0,37.0,40.0,40.0,19.0,25.0,25.0,25.0,25.0,32.0,32.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [224]:
# reformat date and cases - assign to columns

date_cols = total_df.iloc[:, 13:-1].columns

total_df = pd.melt(total_df, id_vars = total_df.iloc[:, 0:13].columns, value_vars = date_cols)

total_df.head()

,Province/State,city_county_state,country,latest_hosp_beds_per_1000,latest_doctors_per_1000,GDP_per_capita,SPAR_index,IVDI_score,city_pop,effective_dist,geo_dist,ann_pass_flux,connected_airports,variable,value
0,0,", ,",San Marino,3.8,6.1468,13536.794829,32.846154,0.696439,811.059130,12.909125,8530.463669,3.312232e+06,54.719512,1/10/20,0.0
1,0,", , Abu Dhabi",United Arab Emirates,1.2,2.3944,75075.257411,94.615385,0.718802,1210.169908,12.417796,5798.434510,9.665304e+06,70.277778,1/10/20,0.0
2,0,", , Al Wusta",Oman,1.6,1.9697,41859.934477,100.000000,0.617149,1085.491700,11.108324,5690.297303,8.373158e+06,72.250000,1/10/20,0.0
3,0,", , Jahra",Kuwait,2.0,2.5789,72897.562345,55.615385,0.668414,1290.116059,13.896034,6180.024430,2.496246e+06,28.187500,1/10/20,0.0
4,0,", , Lopburi Province",Thailand,2.1,0.8096,19051.333382,79.461538,0.687330,1862.299400,7.352859,2074.761769,6.238800e+06,32.937500,1/10/20,0.0


In [225]:
# reorder columns
total_df = total_df.rename(columns = {'latest_hosp_beds_per_1000':'hospital beds',
                                      'latest_doctors_per_1000':'physicians',
                                      'variable':'date',
                                         'value':'confirmed_cases'})

# one-hot encode region and continent

# drop airport_name, _id?

In [226]:
total_df['date'] = pd.to_datetime(total_df['date'], infer_datetime_format=True)


In [227]:
total_df.to_csv('time_series_cases.csv')

In [228]:
total_df.head()

,Province/State,city_county_state,country,hospital beds,physicians,GDP_per_capita,SPAR_index,IVDI_score,city_pop,effective_dist,geo_dist,ann_pass_flux,connected_airports,date,confirmed_cases
0,0,", ,",San Marino,3.8,6.1468,13536.794829,32.846154,0.696439,811.059130,12.909125,8530.463669,3.312232e+06,54.719512,2020-01-10,0.0
1,0,", , Abu Dhabi",United Arab Emirates,1.2,2.3944,75075.257411,94.615385,0.718802,1210.169908,12.417796,5798.434510,9.665304e+06,70.277778,2020-01-10,0.0
2,0,", , Al Wusta",Oman,1.6,1.9697,41859.934477,100.000000,0.617149,1085.491700,11.108324,5690.297303,8.373158e+06,72.250000,2020-01-10,0.0
3,0,", , Jahra",Kuwait,2.0,2.5789,72897.562345,55.615385,0.668414,1290.116059,13.896034,6180.024430,2.496246e+06,28.187500,2020-01-10,0.0
4,0,", , Lopburi Province",Thailand,2.1,0.8096,19051.333382,79.461538,0.687330,1862.299400,7.352859,2074.761769,6.238800e+06,32.937500,2020-01-10,0.0


In [ ]:
# assign multiple numbers of cases (sliding window) per time series as in LSTM class notebook
# https://github.com/learn-co-students/dc-ds-111819/blob/master/module-6/day-8-lstms/LSTMs_enkeboll.ipynb

In [ ]:
# impute model data back to Dec 8th?

In [ ]:
# labels

In [182]:
# closest airports to provinces/locations?

### 11. Add time series from news sources dating back to Dec 8th, date of outbreak 

[China CDC](/Users/flatironschooldc3/Documents/The+Epidemiological+Characteristics+of+an+Outbreak+of+2019+Novel+Coronavirus+Diseases+(COVID-19)+%E2%80%94+China,+2020.pdf)

[Singapore](https://www.bbc.com/news/world-asia-51480613)

[Travel Restrictions](https://www.thinkglobalhealth.org/article/travel-restrictions-china-due-covid-19?utm_source=dailybrief&utm_medium=email&utm_campaign=DailyBrief2020feb20&utm_term=DailyNewsBrief)

In [46]:
# merge new time date stuff from articles and mitigation measures into jhu set
jhu_df = pd.read_csv('jhu_df_for_imputation.csv', index_col = 0)

date_cols = ['12/08/19', '12/11/19', '12/12/19', '12/15/19', '12/17/19','12/18/19', 
                                        '12/19/19', '12/20/19','12/21/19', '12/22/19','12/23/19', '12/24/19', 
                                        '12/25/19', '12/26/19', '12/27/19','12/28/29', '12/29/19', '12/30/19',
                                        '12/31/19']

Hubei_time_list = [1, 
              sum([1,1]), 
              sum([1,1,3]), 
              sum([1,1,3,3]), 
              sum([1,1,3,3,4]), 
              sum([1,1,3,3,4,2]), 
              sum([1,1,3,3,4,2,2]), 
              sum([1,1,3,3,4,2,2,9]), 
              sum([1,1,3,3,4,2,2,9,4]), 
              sum([1,1,3,3,4,2,2,9,4,8]), 
              sum([1,1,3,3,4,2,2,9,4,8,3]), 
              sum([1,1,3,3,4,2,2,9,4,8,3,5]), 
              sum([1,1,3,3,4,2,2,9,4,8,3,5,2]), 
              sum([1,1,3,3,4,2,2,9,4,8,3,5,2,2]), 
              sum([1,1,3,3,4,2,2,9,4,8,3,5,2,2,10]), 
              sum([1,1,3,3,4,2,2,9,4,8,3,5,2,2,10,9]), 
              sum([1,1,3,3,4,2,2,9,4,8,3,5,2,2,10,9,10]), 
              sum([1,1,3,3,4,2,2,9,4,8,3,5,2,2,10,9,10,12]), 
              sum([1,1,3,3,4,2,2,9,4,8,3,5,2,2,10,9,10,12,14])]

jhu_df = pd.concat([jhu_df, pd.DataFrame(columns=date_cols)])

for i, (col, val) in enumerate(list(zip(date_cols, Hubei_time_list))):
    jhu_df.set_value(index=12, col=col, value=val, takeable=False)

new_date_cols = ['1/10/20']
jhu_df = pd.concat([jhu_df, pd.DataFrame(columns=new_date_cols)])

jhu_df[['Province/State', '1/10/20', '1/22/20']].sort_values('Province/State')

states = ['Anhui', 'Beijing', 'Fujian', 'Gansu', 'Chongqing', 'Guangdong', 'Guangxi', 'Guizhou',
         'Hainan', 'Hebei', 'Heilongjiang', 'Henan', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi',
         'Jilin', 'Liaoning', 'Ningxia', 'Qinghai', 'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 
         'Sichuan', 'Xinjiang', 'Yunnan', 'Zhejiang']

time_list_1_10 = [1, 1, 1, 1, 1, 1, 1, 0, 1, 
            1, 1, 1, 1, 1, 0, 1, 1,
            0, 0, 0, 0, 0, 1, 1, 0, 
            1, 0, 1, 1]

for i, (state, val) in enumerate(list(zip(states, time_list_1_10))):
    jhu_df.set_value(index=jhu_df.loc[jhu_df['Province/State'] == state].index[0], col='1/10/20', value=val, takeable=False)

jhu_df.fillna(0, inplace = True)

jhu_df.to_csv('jhu_df_imputed.csv')

jhu_df.head() 

,1/10/20,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,12/08/19,12/11/19,12/12/19,12/15/19,12/17/19,12/18/19,12/19/19,12/20/19,12/21/19,12/22/19,12/23/19,12/24/19,12/25/19,12/26/19,12/27/19,12/28/29,12/29/19,12/30/19,12/31/19,2/1/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/2/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,Lat,Long,Province/State,city_county_state,country,lat_long
0,1,1.0,9.0,15.0,39.0,60.0,70.0,106.0,152.0,200.0,237.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,297.0,830.0,860.0,889.0,910.0,934.0,950.0,962.0,973.0,982.0,986.0,340.0,987.0,988.0,989.0,989.0,989.0,989.0,989.0,989.0,408.0,480.0,530.0,591.0,665.0,733.0,779.0,31.8257,117.2264,Anhui,"Lianhua Community, 合肥政务文化新区, Anhui",China,"31.8257,117.2264"
1,1,14.0,22.0,36.0,41.0,68.0,80.0,91.0,111.0,114.0,139.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,168.0,337.0,342.0,352.0,366.0,372.0,375.0,380.0,381.0,387.0,393.0,191.0,395.0,396.0,399.0,399.0,399.0,400.0,400.0,410.0,212.0,228.0,253.0,274.0,297.0,315.0,326.0,40.1824,116.4142,Beijing,", Changping District, Beijing",China,"40.1824,116.4142"
2,1,6.0,9.0,27.0,57.0,75.0,110.0,132.0,147.0,182.0,211.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,247.0,486.0,505.0,518.0,529.0,537.0,544.0,551.0,553.0,555.0,560.0,300.0,567.0,572.0,573.0,575.0,576.0,576.0,576.0,576.0,337.0,366.0,389.0,411.0,426.0,428.0,468.0,30.0572,107.8740,Chongqing,", Fengdu County, Chongqing",China,"30.0572,107.874"
3,1,1.0,5.0,10.0,18.0,35.0,59.0,80.0,84.0,101.0,120.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144.0,261.0,267.0,272.0,279.0,281.0,285.0,287.0,290.0,292.0,293.0,159.0,293.0,293.0,293.0,293.0,293.0,294.0,294.0,296.0,179.0,194.0,205.0,215.0,224.0,239.0,250.0,26.0789,117.9874,Fujian,"Sanming City, Youxi County, Fujian",China,"26.0789,117.9874"
4,1,0.0,2.0,2.0,4.0,7.0,14.0,19.0,24.0,26.0,29.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40.0,83.0,86.0,87.0,90.0,90.0,90.0,90.0,91.0,91.0,91.0,51.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,55.0,57.0,62.0,62.0,67.0,79.0,83.0,36.0611,103.8343,Gansu,"Guangwumen Subdistrict, Chengguan District, Gansu",China,"36.0611,103.8343"


In [ ]:
# time series?